# Ensemble Retriever（整合檢索器）

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/10-Retriever/03-EnsembleRetriever.ipynb)[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/10-Retriever/03-EnsembleRetriever.ipynb)
## 概述

此筆記本探討在 LangChain 中建立與使用 **EnsembleRetriever**，透過結合多種檢索方法來提升資訊檢索的效果。<br>
EnsembleRetriever 整合了稀疏與稠密檢索演算法的優勢，並可利用權重與執行時設定進行客製化調校以獲得更佳效能。<br>

**重點功能**
1. **整合多種搜尋器**：接受不同類型的搜尋器作為輸入並合併結果。
2. **結果重排序**：使用 [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf)（RRF）演算法對結果重新排序。
3. **混合式搜尋**：主要結合 ```sparse retriever```（例如 BM25）與 ```dense retriever```（例如 embedding 相似度）。

**優勢**
- 稀疏檢索器：擅長以關鍵字為主的查詢。
- 稠密檢索器：擅長以語意相似度為主的查詢。

由於以上互補特性，```EnsembleRetriever``` 能在多種搜尋情境中提供更佳的整體表現。

更多資訊請參考 [LangChain 官方文件](https://python.langchain.com/api_reference/langchain/retrievers.html)



### 目錄

- [概述](#概述)
- [環境設定](#環境設定)
- [建立與設定 Ensemble Retriever](#建立與設定-ensemble-retriever)
- [執行查詢](#執行查詢)
- [變更執行時設定](#變更執行時設定)


### 參考資料

- [LangChain: EnsembleRetriever](https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.ensemble.EnsembleRetriever.html#ensembleretriever)
- [LangChain: BM25Retriever](https://python.langchain.com/api_reference/community/retrievers/langchain_community.retrievers.bm25.BM25Retriever.html)
- [LangChain: ConfigurableField](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.utils.ConfigurableField.html)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [3]:
%%capture --no-stderr
!pip install langchain-opentutorial

In [1]:
# 安裝所需套件
from langchain_opentutorial import package

package.install(
    [
        "langchain_core",       # LangChain 核心功能
        "langchain_community",  # 社群支援的整合套件
        "langchain_openai",     # OpenAI 整合（用於向量嵌入與模型）
        "rank_bm25",            # BM25 排序演算法（資訊檢索用）
    ],
    verbose=False,  # 關閉詳細安裝日誌
    upgrade=False,  # 若套件已安裝則不升級
)


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


In [ ]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "Conversation-With-History",
    }
)

Environment variables have been set successfully.


In [6]:
# Configuration file to manage API keys as environment variables
from dotenv import load_dotenv

# Load API key information
load_dotenv(override=True)

False

## 建立與設定 Ensemble Retriever

**初始化 Ensemble Retriever**  
Ensemble Retriever 結合了兩種不同的資料發現機制：

- **稀疏搜尋（Sparse search）**：使用 **BM25Retriever** 進行關鍵字比對。
- **稠密搜尋（Dense search）**：使用 **FAISS** 搭配 **OpenAI Embedding** 進行語意相似度比對。

初始化 ```EnsembleRetriever``` 時，會將 ```BM25Retriever``` 與 ```FAISS``` 搜尋器結合，並為每個搜尋器設定權重（weights）。

## 稀疏檢索（Sparse Retrieval）與稠密檢索（Dense Retrieval）解釋

在資訊檢索系統中，**稀疏檢索**與**稠密檢索**是兩種常見且互補的搜尋方法。

---

### 1. 稀疏檢索（Sparse Retrieval）
- **原理**：依賴關鍵字匹配，使用稀疏向量（大部分值為 0 的向量）來表示文本。
- **代表技術**：BM25、TF-IDF。
- **特點**：
  - 速度快，對關鍵字精確匹配非常有效。
  - 適合精確搜尋（Exact Match），例如搜尋特定法律條款或專有名詞。
  - 對於同義詞、語意變化的處理能力較弱。
- **範例**：
  - 搜尋「授信條件」，系統會優先返回文件中出現這四個字的段落。

---

### 2. 稠密檢索（Dense Retrieval）
- **原理**：將文本轉換為稠密向量（每個維度都有實數值），通常由深度學習模型（如 Embedding 模型）生成。
- **代表技術**：FAISS、Annoy、HNSW 等向量搜尋工具。
- **特點**：
  - 擅長捕捉語意相似性，即使關鍵字不同，也能找到相關內容。
  - 適合語意搜尋（Semantic Search），例如「貸款條件」也能找到「授信規範」的內容。
  - 計算成本相對較高，需要向量索引與相似度計算。
- **範例**：
  - 搜尋「貸款利率要求」，即使文件中沒有完全相同的字詞，也能找到描述「貸款利息條件」的內容。

---

### 3. 為什麼要混合使用（Hybrid Search）
- **互補性**：
  - 稀疏檢索可確保精確的關鍵字匹配。
  - 稠密檢索可補足語意相關性的不足。
- **應用場景**：
  - 在法律文件搜尋中，混合檢索可同時找出**精確條文**與**語意相關內容**，提升檢索覆蓋率與準確度。

In [1]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 範例文件列表
doc_list = [
    "I like apples",
    "I like apple company",
    "I like apple's iphone",
    "Apple is my favorite company",
    "I like apple's ipad",
    "I like apple's macbook",
]

# 初始化 BM25 檢索器與 FAISS 檢索器
bm25_retriever = BM25Retriever.from_texts(
    doc_list,
)
bm25_retriever.k = 1  # 設定 BM25Retriever 回傳的搜尋結果數量為 1

embedding = OpenAIEmbeddings()  # 啟用 OpenAI 向量嵌入（Embeddings）

faiss_vectorstore = FAISS.from_texts(
    doc_list,
    embedding,
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 1})

# 初始化 Ensemble Retriever（整合檢索器）
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],  # 同時使用 BM25 與 FAISS
    weights=[0.7, 0.3],  # 設定權重（BM25: 0.7、FAISS: 0.3）
)

## 執行查詢（Query Execution）

使用 `ensemble_retriever` 對給定的查詢進行檢索，並比較不同檢索器的結果。

- 呼叫 `ensemble_retriever` 物件的 `get_relevant_documents()` 方法以取得相關文件。

In [ ]:
# 執行檢索並取得結果文件
query = "my favorite fruit is apple"

# 使用整合檢索器（Ensemble Retriever）
ensemble_result = ensemble_retriever.invoke(query)

# 使用 BM25 檢索器
bm25_result = bm25_retriever.invoke(query)

# 使用 FAISS 檢索器
faiss_result = faiss_retriever.invoke(query)

# 輸出檢索到的文件
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

/opt/homebrew/lib/python3.11/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=5689cd5b-392a-4e78-90ba-e84c82574bfd,id=5689cd5b-392a-4e78-90ba-e84c82574bfd; trace=5689cd5b-392a-4e78-90ba-e84c82574bfd,id=291113f2-23ce-45b1-9f0e-01b96cf41b91; trace=5689cd5b-392a-4e78-90ba-e84c82574bfd,id=dfff3ee6-90d0-4dbb-b9a6-0a855eb5daf9
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=541d44ea-973a-43e1-8424-22

[Ensemble Retriever]
Content: Apple is my favorite company

Content: I like apples

[BM25 Retriever]
Content: Apple is my favorite company

[FAISS Retriever]
Content: I like apples



Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=1324bc40-2848-4d2b-9062-0e064f6a238d,id=1324bc40-2848-4d2b-9062-0e064f6a238d


## 檢索結果解釋

你這次查詢 `"my favorite fruit is apple"`，分別使用了三種檢索方式：**Ensemble Retriever（混合檢索）**、**BM25 Retriever（稀疏檢索）**、**FAISS Retriever（稠密檢索）**。以下是結果分析：

---

### 1. BM25 Retriever（稀疏檢索）
- **結果**：`Apple is my favorite company`
- **原因**：
  - BM25 偏向**關鍵字精確匹配**。
  - 查詢與該文件在字面上高度重合（"apple" + "favorite"）。
  - 它不會考慮「fruit」的語意，因此選擇了與公司有關的句子。

---

### 2. FAISS Retriever（稠密檢索）
- **結果**：`I like apples`
- **原因**：
  - FAISS 透過 OpenAI Embeddings 做**語意相似度搜尋**。
  - 它理解「my favorite fruit is apple」與「I like apples」在語意上接近（都在談喜歡水果蘋果）。
  - 因此優先選擇了水果相關的內容，而不是公司。

---

### 3. Ensemble Retriever（混合檢索）
- **結果**：
  1. `Apple is my favorite company`（來自 BM25 權重較高）
  2. `I like apples`（來自 FAISS）
- **原因**：
  - 權重設定為 `BM25: 0.7`、`FAISS: 0.3`，因此更偏向 BM25 排序結果。
  - 第一順位與 BM25 相同，但也納入 FAISS 的高分結果，確保涵蓋語意相關內容。
  - 這種方法同時保留了**關鍵字精準匹配**與**語意關聯性**。

---

### 4. 結論
- **BM25** 適合精確字面搜尋，對語意變化不敏感。
- **FAISS** 擅長語意搜尋，即使字詞不同，也能找到意思接近的內容。
- **Ensemble** 能同時利用兩者優勢，提升搜尋覆蓋率與準確度。

---

💡 **小提示**  
如果想讓搜尋結果更偏向語意判斷，可以調整權重，例如 `weights=[0.4, 0.6]`，讓 FAISS 的影響力更大；反之，想更精確匹配關鍵字，可以增加 BM25 的權重。

In [9]:
# Get the search results document.
query = "Apple company makes my favorite iphone"
ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
faiss_result = faiss_retriever.invoke(query)

# Output the fetched documents.
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: Apple is my favorite company

Content: I like apple's iphone

[BM25 Retriever]
Content: Apple is my favorite company

[FAISS Retriever]
Content: I like apple's iphone



## Change runtime config

You can also change the properties of a retriever at runtime. This is possible using the ```ConfigurableField``` class.

- Define the ```weights``` parameter as a ```ConfigurableField``` object.
  - Set the field's ID to “ensemble_weights”.


## 變更執行時設定（Change runtime config）

在執行過程中，你也可以動態修改檢索器（retriever）的屬性。  
這可以透過 `ConfigurableField` 類別來實現。

---

### 步驟說明
1. 將 `weights` 參數定義為 `ConfigurableField` 物件。
2. 為該欄位設定唯一的識別 ID（此例為 `"ensemble_weights"`）。

In [ ]:
from langchain_core.runnables import ConfigurableField

ensemble_retriever = EnsembleRetriever(
    # 設定檢索器清單，此處使用 bm25_retriever 與 faiss_retriever
    retrievers=[bm25_retriever, faiss_retriever],
).configurable_fields(
    weights=ConfigurableField(
        # 搜尋參數的唯一識別 ID
        id="ensemble_weights",
        # 搜尋參數的名稱
        name="Ensemble Weights",
        # 搜尋參數的描述
        description="Ensemble Weights",
    )
)

- 在搜尋時，透過 `config` 參數指定搜尋設定。
  - 將 `ensemble_weights` 選項的權重設定為 `[1, 0]`，讓 **所有搜尋結果更偏向 BM25 檢索器**。

In [11]:
config = {"configurable": {"ensemble_weights": [1, 0]}}

# Use the config parameter to specify search settings.
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)
docs  # Print the search result, docs.

[Document(metadata={}, page_content='Apple is my favorite company'),
 Document(id='6280c2a3-b58f-474e-aeb6-d480bb44d49e', metadata={}, page_content='I like apples')]

This time, we want all search results to be weighted **more heavily in favor of the FAISS retriever**.

In [12]:
config = {"configurable": {"ensemble_weights": [0, 1]}}

# Use the config parameter to specify search settings.
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)
docs  # Print the search result, docs.

[Document(id='6280c2a3-b58f-474e-aeb6-d480bb44d49e', metadata={}, page_content='I like apples'),
 Document(metadata={}, page_content='Apple is my favorite company')]